### PyWren RISECamp, 2017

Welcome to the hands-on tutorial for PyWren.

This tutorial consists of a set of exercises that will have you working directly with PyWren:
- simple matrix multiplication
- data analysis on a wikipedia dataset
- some machine learning algorithms (Eric's) 


## 0. Hello World

First, let's write a simple hello program to test out PyWren.



In [ ]:
# some libraries that are useful for this tutorial
import sys
if '/pywren-setup/' not in sys.path:
    sys.path.insert(0, '/pywren-setup/')

from training import *

# first we need to load PyWren and creates an executor instance
import pywren
pwex = pywren.default_executor()

### 0.1. call_async() -- our single invocation API
We can use the `call_async()` API on pywren executor to run the function in the cloud
The workflow is pretty simple and looks like this:

```python
def my_func(param):
    # do something
    return some_result
    
handler = pwex.call_async(my_func, param)
result = handler.result()
```

**Exercise**: modify the following code block to run hello world with pywren

**TODO: I think we need helper function to test against the output. This gives attendees more incentive to get things right.**

In [ ]:
# first we need a basic hello world function
def hello_world(param):
    if param == 42:
        return "hello world!"

future = pwex.call_async()
# on success, this line should print out "hello world"
check_result_1(future.result())

### 0.2. map() -- parallel execution in the cloud
The above example runs a single function in the cloud.
Now PyWren also has a `map()` API that allows users to run a single function with multiple parameters:

```python
handlers = pwex.map(my_func, param_list)
pywren.wait(handlers)

results = [h.result() for h in handlers]
```

**Exercise**: modify the following code block to print "hello world"

In [ ]:
# do not modify code here
def hello_world(param):
    if param == 1:
        return "hello"
    if param == 2:
        return "world!"
# do not modify code above

param_list = []
futures = pwex.call_async(hello_world, None)

results = [f.result() for f in futures] 
check_result_2(" ".join(results))

### 0.3. wait() API and multiple jobs

### 0.4. Visualization and Debugging
From the talk, you have already heard what happens behind every PyWren execution. Let's see it for real!

TODO: maybe we can use plotting for a bigger job. This one is not very interesting with only two tasks. - qifan

**Exercise**: inspect PyWren's execution by running the plotting code below

In [ ]:
plot_pywren_execution(futures)

Another tool you can use is to print latest CloudWatch logs which could tell you about the latest Lambda execution.  

In [ ]:
!pywren print_latest_logs

This concludes our startup section. You can find more documentation on PyWren APIs and usages at http://pywren.io/

## 1. Matrix Multiplication

One nice thing about PyWren is it allows users to integrate existing python libraries easily.
For the following exercise, we are going to use some popular python libraries, e.g., NumPy, to work on some matrix multiplication problems.

In [ ]:
import numpy as np

def my_function(b):
    x = np.random.normal(0, b, 1024)
    A = np.random.normal(0, b, (1024, 1024))
    return np.dot(A, x)

pwex = pywren.default_executor()
res = pwex.map(my_function, np.linspace(0.1, 10, 100))


## 2. Data Analytics with Wikipedia Dataset

In [ ]:
filenames = list_keys_with_prefix(rise_camp_bucket, "wikistats_20090505_restricted-01/")
print(len(filenames))

In [ ]:
def take5(filename):
    data = pywren_read_data(rise_camp_bucket, filename)
    result = data.split("\n")[:5]
    return result

future = pwex.call_async(take, filenames[0])
for x in future.result():
    print(x)

In [ ]:
def count(filename):
    data = pywren_read_data(rise_camp_bucket, filename)
    return (len(data.split("\n")) if data else 0)    

futures = pwex.map(count, filenames)
pywren.wait(futures)

result = sum([f.result() for f in futures])
print(result)

In [ ]:
from itertools import groupby
from operator import itemgetter
from functools import reduce

def aggregate_count(key_value_list):
    def reduce_f(obj1, obj2):
        return(obj1[0], obj1[1] + obj2[1])
    counts = [reduce(reduce_f, group) for _, group 
          in groupby(sorted(key_value_list), key=itemgetter(0))]
    
    return counts

def english_page_count(filename):
    data = pywren_read_data(rise_camp_bucket, filename)
    # filter out the english pages
    en_pages = [d for d in data.split("\n") 
                if len(d.split(" ")) >= 4 and d.split(" ")[1] == "en"]
    # projection to create (date, pagecount) pairs
    en_kvpair_list = [(p.split(" ")[0][:8], int(p.split(" ")[3])) for p in en_pages]

    return aggregate_count(en_kvpair_list)
    
futures = pwex.map(english_page_count, filenames)
pywren.wait(futures)

results = [f.result() for f in futures]
en_page_counts_by_date = aggregate_count([x for y in results for x in y])
print(en_page_counts_by_date)

## 3. Some Machine Learning